#### Dictionary of Atomic Masses

In [3]:
atomicMasses = {
    (21,7): 25.230,     (21,8): 8.062,      (21,9): -0.0476,    (21,10): -5.73172,  (21,11): -2.1843,   (21,12): 10.912,
    (22,6): 52.600,     (22,7): 32.080,     (22,8): 9.280,      (22,9): 2.794,      (22,10): -8.02435,  (22,11): -5.1822, 
    (22,12): -0.3968,   (22,13): 18.180,    (22,14): 32.160,    (23,7): 37.700,     (23,8): 14.620,     (23,9): 3.330, 
    (23,10): -5.1537,   (23,11): -9.52950,  (23,12): -5.4727,   (23,13): 6.767,     (23,14): 23.770,    (24,8): 19.000, 
    (24,9): 7.540,      (24,10): -5.948,    (24,11): -8.41762,  (24,12): -13.93340, (24,13): -0.055,    (24,14): 10.755,
    (25,9): 11.270,     (25,10): -2.060,    (25,11): -9.3575,   (25,12): -13.19275, (25,13): -8.9158,   (25,14): 3.825,
    (26,9): 18.290,     (26,10): 0.430,     (26,11): -6.902,    (26,12): -16.21451, (26,13): -12.21032, (26,14): -7.145, 
    (26,15): 10.970,    (27,9): 25.100,     (27,10): 7.090,     (27,11): -5.580,    (27,12): -14.58654, (27,13): -17.19686, 
    (27,14): -12.38503, (27,15): -0.750,    (27,16): 17.510,    (28,10): 11.280,    (28,11): -1.030,    (28,12): -15.0188, 
    (28,13): -16.85058, (28,14): -21.49283, (28,15): -7.161,    (28,16): 4.070,     (29,10): 18.000,    (29,11): 2.620, 
    (29,12): -10.660,   (29,13): -18.2155,  (29,14): -21.89506, (29,15): -16.9519,  (29,16): -3.160,    (30,10): 22.200, 
    (30,11): 8.590,     (30,12): -8.880,    (30,13): -15.872,   (30,14): -24.43292, (30,15): -20.2006,  (30,16): -14.063,
    (31,11): 12.660,    (31,12): -3.220,    (31,13): -14.954,   (31,14): -22.94899, (31,15): -24.4410,  (31,16): -19.0449, 
    (31,17): -7.060,    (31,18): 11.300,    (32,11): 18.300,    (32,12): -0.800,    (32,13): -11.060,   (32,14): -24.0809, 
    (32,15): -24.3053,  (32,16): -26.01594, (32,17): -13.331,   (32,18): -2.180,    (33,11): 25.500,    (33,12): 5.200, 
    (33,13): -8.500,    (33,14): -20.492,   (33,15): -26.3377,  (33,16): -26.58620, (33,17): -21.0035,  (33,18): -9.380,
    (34,11): 32.500,    (34,12): 8.500,     (34,13): -2.860,    (34,14): -19.957,   (34,15): -24.558,   (34,16): -29.93181, 
    (34,17): -24.43961, (34,18): -18.378,   (35,11): 41.200,    (35,12): 16.300,    (35,13): -0.060,    (35,14): -14.360, 
    (35,15): -24.8576,  (35,16): -28.84633, (35,17): -29.01351, (35,18): -23.0482,  (35,19): -11.167,   (35,20): 4.440
    }

#### Function: atomicMass

In [4]:
# Begin atomicMass
# Returns atomic mass excess in MeV.
#  A   is nuclear mass number. Can be any data type that will cast to int.
#  Z   is proton number. Can be any data type that will cast to int. 
def atomicMass(A,Z):
    try:
        A = int(A)
    except:
        print ('\"{}\" is invalid input for A in mass lookup.'.format(A))
        return None
    try:
        Z = int(Z)
    except:
        print ('\"{}\" is invalid input for Z in mass lookup.'.format(Z))
        return None
    
    M = atomicMasses.get((A,Z))
    if M == None:
        print ('No mass found for A,Z={},{}.'.format(A,Z))
    
    return M 
# End atomicMasses

#### Function: avgWeight

In [5]:
# Begin avgWeight
# Computes the average statistical weight for a range of states.
# First finds partial prtition function for a given range then sums spin degeneracies of computed states in that
# range and divides by that sum. Computed states should then be multiplied by their degeneracy and the results from
# avgWeight(). 
#  T         float  temperature (MeV). May be scalar or array.
#  lptName   str    file name for lpt.
#  AZ        str    atomic mass, atomic number
#  channel   str    reaction channel
#  nucDir    str    directory containing the 'overlaps' directory
#  Emin      float  energy minimum of accepted states
#  Emax      float  energy maximum of accepted states
#  Tz        float  z-projection of isospin. May be neglected if lpt file contains no states with bad isospin. 
def avgWeight(T,lptName,AZ,channel,Emin=0,Emax=float('inf'),nucDir='./',Tz=None):
    if not lptName.starswith(nucDir):
        lptName = nucDir + lptName
    
    Z = int(AZ.split(',')[1])
    
    Gp = partfcnPartial(T,lptName,Emin=Emin,Emax=Emax,Tz=Tz)
    
    spinsum = 0
    for dao in os.listdir(nucDir + 'overlaps/A,Z={}'.format(AZ)):
        if os.path.splitext(dao)[1] == '.dao':
            if dao.split()[0] == channel and dao.split()[1].endswith(str(Z)):
                J = float(dao.split()[3].split('=')[1])
                T = float(dao.split()[4].split('=')[1])
                n = dao.split()[5].split('=')[1].split('.')[0]
                E = getEnergy(Tz,J,T,n,lptName=lptName)
                
                if Emin <= E < Emax:
                    spinsum = spinsum + (2*J) + 1

    weight = Gp/spinsum
    return weight 
# End avgWeight

#### Function: BGT

In [6]:
# Begin BGT
# Takes log(ft) value and BF, returns BGT. If Ji and Jf are specified, alos returns reverse reaction BGT.
#  logft  float  measured log(ft) value 
#  Ji     float  initial state spin
#  Jf     float  final state spin 
#  BF     float  Fermi transition strength
def BGT(logft,Ji=None,Jf=None,BF=0.):
    ft = 10**(logft)
    
    # BGT for forward reaction
    BGTf = (BtoFT / ft - BF) / aVectoVec
    
    # FFN values of BtoFT and aVectoVec in following line.
    #BGTf = 10**(3.596)*(100**(-logft)-BF*10**(-3.791))
    
    # BGT for reverse reaction
    if Ji != None and Jf != None:
        BGTr = BGTf * (2.*Ji+1) / (2.*Jf+1)
    else:
        BGTr = None
    
    #print '\nft: {}\nJi: {}\nJf: {}\nBF: {}'.format(logft,Ji,Jf,BF)
    
    return (BGTf,BGTr)
# End BGT

#### Function BGTFFN

In [ ]:
# Begin BGTFFN
# Takes log(ft) value and BF, returns BGT using FFN values of weak constants. If Ji and Jf are specified,
# also returns reverse reaction BGT. 
#  logft  float  measured log(ft) value
#  Ji     float  initial state spin
#  Jf     float  final state spin 
#  BF     float  Fermi transition strength 
def BGTFFN(logft,Ji=None,Jf=None,BF=0.):
    ft = 10**(logft)
    
    # BGT for forward reaction
    BGTf = 10**(3.596)*(10**(-logft)-BF*10**(-3.791))
    
    # BGT for reverse reaction
    if Ji != None and Jf != None:
        BGTr = BGTf * (2.*Ji+1) / (2.*Jf+1)
    else:
        BGTr = None 
        

#### Function: userFloat

In [2]:
# Begin userFloat
# Gets input from the user and converts to appropriate format.
#   prompt    is the prompt given to the user
#   notes     is the parenthetical, e.g., units
#   default   is an optinoal default value 
def userFloat(prompt,notes=None,default=None):
    prompt = '\n' + prompt
    if notes:
        if default:
            other = ' ({}, default {}): '.format(notes,default)
        else:
            other = ' ({}): '.format(notes) 
    else:
        if default:
            other = ' (default {}): '.format(default)
        else:
            other = ': '
    
    if default:
        tempInput = input(prompt+other)
        try:
            tempInput = float(tempInput)
        except:
            print ('Using default.')
            tempInput = float(default)
    else:
        while TRue:
            tempInput = input(prompt+tempInput)
            try:
                tempInput = float(tempInput)
                break
            except:
                print ('Invalid input,')
                
    return tempInput
# End userFloat 